In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, accuracy_score
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Load the Wisconsin Breast Cancer dataset
file_path = r"C:\Users\Shaurya\Downloads\data.csv"
df = pd.read_csv(file_path)

In [3]:
# Drop unnecessary columns
df = df.drop(columns=['id', 'Unnamed: 32'])

In [4]:
# Encode the 'diagnosis' column (M=1, B=0)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['diagnosis'] = label_encoder.fit_transform(df['diagnosis'])

In [5]:
# Handle missing values by dropping rows with null values
df = df.dropna()

In [6]:
# Split the dataset into features (X) and target variable (y)
X = df.drop(columns=['diagnosis'])
y = df['diagnosis']

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Apply PCA
pca = PCA(n_components=10)  # You can adjust the number of components
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [10]:
# Build the Neural Network model
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=X_train_pca.shape[1]))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(X_train_pca, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
12/12 [==============================] - 1s 18ms/step - loss: 1.0499 - accuracy: 0.1951 - val_loss: 1.0093 - val_accuracy: 0.1538
Epoch 2/50
12/12 [==============================] - 0s 4ms/step - loss: 0.8883 - accuracy: 0.2885 - val_loss: 0.8543 - val_accuracy: 0.3077
Epoch 3/50
12/12 [==============================] - 0s 4ms/step - loss: 0.7588 - accuracy: 0.4258 - val_loss: 0.7401 - val_accuracy: 0.4725
Epoch 4/50
12/12 [==============================] - 0s 5ms/step - loss: 0.6594 - accuracy: 0.5742 - val_loss: 0.6533 - val_accuracy: 0.5934
Epoch 5/50
12/12 [==============================] - 0s 4ms/step - loss: 0.5796 - accuracy: 0.7005 - val_loss: 0.5862 - val_accuracy: 0.7033
Epoch 6/50
12/12 [==============================] - 0s 4ms/step - loss: 0.5146 - accuracy: 0.7830 - val_loss: 0.5306 - val_accuracy: 0.7692
Epoch 7/50
12/12 [==============================] - 0s 4ms/step - loss: 0.4586 - accuracy: 0.8516 - val_loss: 0.4815 - val_accuracy: 0.8132
Epoch 8/50
12/12 [=

In [13]:
# Evaluate the model on the test set
y_pred_prob = model.predict(X_test_pca)
y_pred = (y_pred_prob > 0.5).astype(int)

4/4 [==============================] - 0s 2ms/step


In [14]:
# Print accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

Accuracy: 0.9737
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98        71
           1       0.98      0.95      0.96        43

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

